In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [3]:
import yaml

In [4]:
from pprint import pprint

In [5]:
import json

In [6]:
from langchain.prompts import HumanMessagePromptTemplate

In [7]:
def get_fore_context(inputs):
    return inputs[: inputs.find("{end token}")].replace("{start token}","")

In [8]:
inputs = '{start token}import numpy as np\nimport scipy as sp\n{end token}def hello_world():\n    print("Hello world"){middle token}'
data = {"inputs": inputs, "parameters": {"max_new_tokens": 256}}

In [9]:
get_fore_context(inputs)

'import numpy as np\nimport scipy as sp\n'

In [10]:
class LLM:
    verbose = False
    def __init__(self, conf_file) -> None:
        with open(conf_file, "r", encoding="utf-8") as f:
            conf = yaml.safe_load(f.read())
        OPENAI_API_KEY = conf["OPENAI_API_KEY"]
        OPENAI_API_BASE = conf["OPENAI_API_BASE"]
        OPENAI_API_VERSION = conf["OPENAI_API_VERSION"]
        DEPLOYMENT = conf["DEPLOYMENT"]
        self.llm = AzureChatOpenAI(
            openai_api_type="azure",
            openai_api_version=OPENAI_API_VERSION,
            openai_api_base=OPENAI_API_BASE,
            openai_api_key=OPENAI_API_KEY,
            deployment_name=DEPLOYMENT,
            temperature=0,
        )

    def complete(self, query):
        """Answer to a query."""
        system_setting = SystemMessage(
            content="You help to complete the user's code.")
        
        message = HumanMessage(content=query)
        return self.llm.invoke([system_setting, message]).content
    
    def complete_code(self, code_context):
        """Take the input from the request and output.

        args:
            code_context(str): the code_context

        return(dict): the response
        """
        fore_context = get_fore_context(code_context)
        system_setting = SystemMessage(
            content="You are a code autocompleter.")
        prompt = f"""
        Please complete code for the following code. Make code completion after the end token.
        \n\n
        {fore_context}
        """
        message = HumanMessage(content=prompt)
        if LLM.verbose:
            from pprint import pprint
            pprint(prompt)
        return self.llm.invoke([system_setting, message]).content

llm = LLM(".env-35-16k.yml")

In [12]:
LLM.verbose = True
pprint(llm.complete_code(inputs))

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='ail-openai-voc-positive-reviews.openai.azure.com', port=443): Max retries exceeded with url: //openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-05-15 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fd9c152c460>: Failed to resolve 'ail-openai-voc-positive-reviews.openai.azure.com' ([Errno 8] nodename nor servname provided, or not known)")).


('\n'
 '        Please complete code for the following code. Make code completion '
 'after the end token.\n'
 '        \n'
 '\n'
 '\n'
 '        import numpy as np\n'
 'import scipy as sp\n'
 '\n'
 '        ')


APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='ail-openai-voc-positive-reviews.openai.azure.com', port=443): Max retries exceeded with url: //openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-05-15 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fd9c152cc70>: Failed to resolve 'ail-openai-voc-positive-reviews.openai.azure.com' ([Errno 8] nodename nor servname provided, or not known)"))